# Vessel Visualization

We  imports necessary modules, reads two CSV files (rotterdam_hamburg.csv & felixstowe_rotterdam.csv)into pandas DataFrames, applies a cleaning function from the predictor.clean module to each DataFrame, and then concatenates the two cleaned DataFrames into one combined DataFrame.

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
from sklearn.model_selection import train_test_split


import predictor.clean as clean
import predictor.plot as plot


df_rtm_ham_raw = pd.read_csv(
    '../resources/rotterdam_hamburg/rotterdam_hamburg.csv', quotechar="'")
df_fxt_rtm_raw = pd.read_csv(
    '../resources/felixstowe_rotterdam/felixstowe_rotterdam.csv', quotechar="'")
df_combined_raw = pd.concat(
    [df_rtm_ham_raw, df_fxt_rtm_raw], ignore_index=True)


df_rtm_ham = clean.clean_up(df_rtm_ham_raw)
df_fxt_rtm = clean.clean_up(df_fxt_rtm_raw)
df_combined = pd.concat([df_rtm_ham, df_fxt_rtm], ignore_index=True)

print(
    f"df_rtm_ham: {df_rtm_ham_raw.shape} to {df_rtm_ham.shape} ({df_rtm_ham_raw.shape[0] - df_rtm_ham.shape[0]})")
print(
    f"df_fxt_rtm: {df_fxt_rtm_raw.shape} to {df_fxt_rtm.shape} ({df_fxt_rtm_raw.shape[0] - df_fxt_rtm.shape[0]})")




This code presents a GUI for visualizing vessel movement, speed, and trajectory using data from a CSV file. The GUI, created with PySimpleGUI, enables users to select a CSV file and choose from three plot options: "Plot Vessel Movement," "Plot Vessel Speed," and "Plot Vessel Trajectory." The code utilizes libraries like pandas, seaborn, matplotlib, and plotly.express for data handling and visualization. Error handling is included to address empty or inaccessible CSV files, with corresponding error messages displayed. The GUI window remains active until the user selects the "Exit" button or closes the window. 

In [ ]:
import PySimpleGUI as sg
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import tempfile
from predictor.plot import *

# Set the GUI theme
sg.theme('DarkBlack')

# Define the GUI layout
layout = [
    [sg.Text("Select a CSV file:")],
    [sg.Input(key="-FILE-"), sg.FileBrowse(file_types=(("CSV Files", "*.csv"),))],
    [sg.Button("Plot Vessel Movement"), sg.Button("Plot Vessel Speed"), sg.Button("Plot Vessel Trajectory"), sg.Button("Exit")],
    [sg.Output(size=(80, 20))]
]

# Create the GUI window
window = sg.Window("Vessel Visualization", layout)

# Event loop to handle GUI events
while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == "Exit":
        break

    if event == "Plot Vessel Movement":
        # Retrieve the selected CSV file
        filename = values["-FILE-"]
        if filename:
            try:
                data = pd.read_csv(filename)
                plot_vessel_movement(data)  # Plot vessel movement
            except pd.errors.EmptyDataError:
                sg.popup('The selected CSV file is empty.')
        else:
            sg.popup('Please select a CSV file.')

    if event == "Plot Vessel Speed":
        # Retrieve the selected CSV file
        filename = values["-FILE-"]
        if filename:
            try:
                data = pd.read_csv(filename)
                plot_vessel_speed(data)  # Plot vessel speed
            except pd.errors.EmptyDataError:
                sg.popup('The selected CSV file is empty.')
        else:
            sg.popup('Please select a CSV file.')

    if event == "Plot Vessel Trajectory":
        # Retrieve the selected CSV file
        filename = values["-FILE-"]
        if filename:
            try:
                data = pd.read_csv(filename)

                fig = px.scatter_mapbox(df_combined.sample(n=20000), lat="Latitude", lon="Longitude",
                             color_discrete_sequence=["fuchsia"], zoom=8, height=None)
                fig.update_layout(mapbox_style="open-street-map")
                fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
                fig.show()

                # Save the plot as a temporary image file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
                    filename = tmpfile.name
                    plt.savefig(filename)

                    gui.gui_func(filename, name="Vessel Trajectory")
                    
            except pd.errors.EmptyDataError:
                sg.popup('The selected CSV file is empty.')
        else:
            sg.popup('Please select a CSV file.')

window.close()
